In [1]:
import branca
import dask.distributed
import folium
import folium.plugins
import geopandas as gpd
import numpy as np
import rasterio as rio
import rasterio.features
import shapely.geometry
import xrspatial.multispectral as ms
from branca.element import Element, Figure
from IPython.display import HTML, display
from odc.stac import configure_rio, stac_load
from pystac_client import Client

from utils import convert_bounds, image_on_map

/tmp/ipykernel_47603/1540613272.py:5: UserWarning: Shapely 2.0 is installed, but because PyGEOS is also installed, GeoPandas will still use PyGEOS by default for now. To force to use and test Shapely 2.0, you have to set the environment variable USE_PYGEOS=0. You can do this before starting the Python process, or in your code before importing geopandas:

import os
os.environ['USE_PYGEOS'] = '0'
import geopandas

In a future release, GeoPandas will switch to using Shapely by default. If you are using PyGEOS directly (calling PyGEOS functions on geometries from GeoPandas), this will then stop working and you are encouraged to migrate from PyGEOS to Shapely 2.0 (https://shapely.readthedocs.io/en/latest/migration_pygeos.html).
  import geopandas as gpd


In [2]:
# stack configuration
cfg = {
    "sentinel-s2-l2a-cogs": {
        "assets": {
            "*": {"data_type": "uint16", "nodata": 0},
            "SCL": {"data_type": "uint8", "nodata": 0},
            "visual": {"data_type": "uint8", "nodata": 0},
        },
        "aliases": {"red": "B04", "green": "B03", "blue": "B02"},
    },
    "*": {"warnings": "ignore"},
}

In [29]:
# start the dask cluster
client = dask.distributed.Client()
configure_rio(cloud_defaults=True, aws={"aws_unsigned": True}, client=client)
client

/home/biel/miniconda3/envs/geo/lib/python3.10/site-packages/distributed/node.py:182: UserWarning: Port 8787 is already in use.
Perhaps you already have a cluster running?
Hosting the HTTP server on port 39131 instead
  warnings.warn(
2023-06-08 10:59:30,491 - distributed.nanny - ERROR - Failed to initialize worker
Traceback (most recent call last):
  File "/home/biel/miniconda3/envs/geo/lib/python3.10/site-packages/distributed/nanny.py", line 926, in run
    worker = worker_factory()
  File "/home/biel/miniconda3/envs/geo/lib/python3.10/site-packages/distributed/worker.py", line 756, in __init__
    self.memory_manager = WorkerMemoryManager(
  File "/home/biel/miniconda3/envs/geo/lib/python3.10/site-packages/distributed/worker_memory.py", line 166, in __init__
    self.data = SpillBuffer(
  File "/home/biel/miniconda3/envs/geo/lib/python3.10/site-packages/distributed/spill.py", line 95, in __init__
    slow = Slow(spill_directory, max_spill)
  File "/home/biel/miniconda3/envs/geo/lib/p

Unexpected exception formatting exception. Falling back to standard exception


Traceback (most recent call last):
  File "/home/biel/miniconda3/envs/geo/lib/python3.10/site-packages/IPython/core/interactiveshell.py", line 3505, in run_code
    if async_:
  File "/tmp/ipykernel_47603/1817239109.py", line 2, in <module>
    client = dask.distributed.Client()
  File "/home/biel/miniconda3/envs/geo/lib/python3.10/site-packages/distributed/client.py", line 1008, in __init__
    }
  File "/home/biel/miniconda3/envs/geo/lib/python3.10/site-packages/distributed/client.py", line 1205, in start
    self._loop_runner.start()
  File "/home/biel/miniconda3/envs/geo/lib/python3.10/site-packages/distributed/utils.py", line 416, in sync
    if error:
  File "/home/biel/miniconda3/envs/geo/lib/python3.10/site-packages/distributed/utils.py", line 389, in f
    future = wait_for(future, callback_timeout)
  File "/home/biel/miniconda3/envs/geo/lib/python3.10/site-packages/tornado/gen.py", line 769, in run
    # Save the exception for later. It's important that
  File "/home/biel/min

# Select a bounding box for your query
Use the rectangle tool and then click on it to get the recatngle json

In [4]:
fig = branca.element.Figure(width="600px", height="600px")
m = folium.Map()
fig.add_child(m)
draw = folium.plugins.Draw(
    position="topleft",
    draw_options={"polyline": {"allowIntersection": False}},
    edit_options={"poly": {"allowIntersection": False}},
).add_to(m)
fig

In [ ]:
# paste here the bbox json
aoi = {
    "type": "Polygon",
    "coordinates": [
        [
            [-61.259766, -64.54844],
            [-61.259766, -61.648162],
            [-53.085938, -61.648162],
            [-53.085938, -64.54844],
            [-61.259766, -64.54844],
        ]
    ],
}
bbox = rasterio.features.bounds(aoi)

In [28]:
catalog = Client.open("https://earth-search.aws.element84.com/v0")

query = catalog.search(
    collections=["sentinel-s2-l2a-cogs"],
    datetime="2023-4",  # from / to
    limit=100,
    bbox=bbox,
    query={"eo:cloud_cover": {"lt": 0.1}},
)

items = list(query.get_items())
print(f"Found: {len(items)} datasets")

# Convert STAC items into a GeoJSON FeatureCollection
stac_json = query.get_all_items_as_dict()

Exception: invalid datetime component: 2023-4

Show on a map the foodprints of the found datasets 

In [ ]:
# https://github.com/python-visualization/folium/issues/1501
gdf = gpd.GeoDataFrame.from_features(stac_json, "epsg:4326")

# Compute granule id from components
gdf["granule"] = (
    gdf["sentinel:utm_zone"].apply(lambda x: f"{x:02d}")
    + gdf["sentinel:latitude_band"]
    + gdf["sentinel:grid_square"]
)

fig = Figure(width="600px", height="600px")
map1 = folium.Map()
fig.add_child(map1)

folium.GeoJson(
    shapely.geometry.box(*bbox),
    style_function=lambda x: dict(fill=False, weight=1, opacity=0.7, color="olive"),
    name="Query",
).add_to(map1)

gdf.explore(
    "granule",
    categorical=True,
    tooltip=[
        "granule",
        "datetime",
        "sentinel:data_coverage",
        "eo:cloud_cover",
    ],
    popup=True,
    style_kwds=dict(fillOpacity=0.1, width=2),
    name="STAC",
    m=map1,
)

map1.fit_bounds(bounds=convert_bounds(gdf.unary_union.bounds))
folium.LayerControl().add_to(map1)

display(fig)

In [ ]:
it = [i for i in items if "20EPQ" in i.id]

In [ ]:
it[1]

In [ ]:
scene = stac_load(
    [it[1]],
    bands=["B04", "B03", "B02"],
    crs="epsg:3857",
    resolution=50,
    chunks={}
)

In [ ]:
scene = scene.where(lambda x: x > 0, other=np.nan).to_array("band").squeeze("time").chunk("auto")

In [ ]:
scene

In [ ]:
image_on_map(ms.true_color(*scene).compute(), bbox)

In [ ]:
# Since we will plot it on a map we need to use `EPSG:3857` projection
crs = "epsg:3857"

data = stac_load(
    items,
    bands=["B04", "B03", "B02"],
    crs=crs,
    resolution=10,
    chunks={},  # <-- use Dask
    # stac_cfg=cfg,
    # bbox=bbox,
)

data

In [ ]:
data = (
    data.where(lambda x: x > 0, other=np.nan).to_array("band").chunk("auto")
)  # sentinel-2 uses 0 as nodata
data

Inspect one of the scenes on the map

In [ ]:
# use ms.true_color to convert RGB chanels and improve the color
selection = data.isel(time=1)

In [ ]:
selection

In [ ]:
image_on_map(ms.true_color(*selection).compute(), bbox)
m

In [ ]:
ms.true_color(*selection).plot.imshow()

Compute the cloudless mossaic by taking the median of each pixel across time

In [ ]:
%%time
median = data.median(dim="time").compute()

In [ ]:
image = ms.true_color(*median)  # expects red, green, blue DataArrays

In [ ]:
import matplotlib.pyplot as plt

fig, ax = plt.subplots(figsize=(12, 12))

ax.set_axis_off()
image.plot.imshow(ax=ax);

In [ ]:
fig = Figure(width="600px", height="600px")
m = folium.Map()
fig.add_child(m)
image.odc.add_to(m)

m.fit_bounds(bounds=convert_bounds(gdf.unary_union.bounds))
folium.LayerControl().add_to(m)
m